# Home
> home

- toc: false 
- badges: true
- comments: false
- categories: []
- image: images/evolucion-por-provincia.png
- home: true
- hide: true

In [1]:
#hide
import arg_data

df = arg_data.get_arg_df()
cnt_by_place = df.groupby('infered_place').infected.sum().sort_values(ascending=False)

Skipping 15032020-nuevo-coronavirus-covid-19_0.pdf


In [2]:
#hide
import altair as alt

def plot_evolution(selected_places=None, xaxis='date', xaxis_title='Fecha'):
    selection = alt.selection_multi(
        fields=['infered_place'], bind='legend', init=[{'infered_place': c} for c in (selected_places or [])]
    )
    
    return (
        alt.Chart(df)
           .mark_line(point=True, radius=150)
           .encode(
               x=alt.X(xaxis, axis=alt.Axis(title=xaxis_title)),
               y=alt.Y('cum_infected', scale=alt.Scale(type='log'), axis=alt.Axis(title='Casos confirmados')),
               color=alt.Color('infered_place', sort=list(cnt_by_place.index)),
               tooltip=list(df.columns), 
               opacity=alt.condition(selection, alt.value(.8), alt.value(.05))
            ).add_selection(selection)
             .configure_point(size=200)
             .properties(width=450, height=400)
             .configure_title(align='center', )
    )
    

### Evolucion en el tiempo de la cantidad de casos totales

Nota de color: **Los graficos se actualizan solos!!**. Una vez por hora se corre el script que busca la data y arma los graficos via una [action de github](https://github.com/elsonidoq/covid-19-argentina/actions)

In [3]:
#hide_input
plot_evolution(cnt_by_place[cnt_by_place >=10].index.tolist())

alt.Chart(...)

Haciendo **Shift+Click** en las provincias que se ven a la derecha, podemos agregar o quitar provincias al gráfico

Esta página es un experimento para evaluar la utilidad de la informacion que se encuentra disponible en el sitio https://www.argentina.gob.ar/coronavirus/informe-diario para tener una vision general de lo que pasa en el pais